Deep Learning: Practical session

## global settings

For faster processing, make sure to set in *Runtime > Change runtime type*, *hardware accelerator* to *GPU*


##Usefull packages

### Tensorflow and Keras

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

print('Tensorflow version:')
!python3 -c 'import tensorflow as tf; print(tf.__version__)'  # for Python 3

In [3]:
import datetime, os
import numpy as np
import matplotlib.pyplot as plt
import glob

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, InputLayer
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.utils import to_categorical
precision, recall = Precision(), Recall()

## Downloading database

Follow the instructions

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

In [5]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
googledrive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
id = '17T6-1E6Nk37O6S6sUhprQvTfLYmXQb-b'
download = googledrive.CreateFile({'id': id})
download.GetContentFile('2021_fruits_dataset.zip')
!unzip '2021_fruits_dataset.zip'

# Database

In [8]:
class_names = ['apple', 'banana', 'cherry', 'grape', 'onion', 'peach', 'pear', 'pepper', 'plum', 'potato', 'tomato']

In [9]:
# rescale images to range [0; 1]
def im_preprocess(im):

    if im.max() > 1.:
        # to float instead of uint8
        im = np.divide(im, 255., dtype=np.float16)

    assert im.max() <= 1.

    return im

 
def get_data(dataset, factor_downsample = 1):
    """
    :param factor_downsample: skips images to reduce dataset size
    """

    assert dataset in ['training', 'validation', 'testing']

    print(f'Loading {dataset}-dataset')

    if dataset == 'training':
        home_folder = '2021_fruits_dataset/1training'
    elif dataset == 'validation':
        home_folder = '2021_fruits_dataset/2validation'
    else:
        home_folder = '2021_fruits_dataset/3test'

    x = []
    y = []

    folder_list = sorted(os.listdir(home_folder))

    for folder in folder_list:
        folder_lower = folder.lower()

        if 'apple' == folder_lower[:5]:            
            y_index = class_names.index('apple')
        
        elif 'banana' in folder_lower:
            y_index = class_names.index('banana')
            
        elif 'cherry' in folder_lower[:6]:
            y_index = class_names.index('cherry')
            
        elif 'grape ' in folder_lower:
            y_index = class_names.index('grape')
            
        elif 'onion' in folder_lower:
            y_index = class_names.index('onion')
            
        elif 'peach' in folder_lower:
            y_index = class_names.index('peach')
            
        elif 'pear' in folder_lower:
            y_index = class_names.index('pear')

        elif 'pepper' in folder_lower:
            y_index = class_names.index('pepper')
            
        elif 'plum' in folder_lower:
            y_index = class_names.index('plum')
            
        elif 'potato' in folder_lower:
            y_index = class_names.index('potato')
            
        elif 'tomato' in folder_lower:
            y_index = class_names.index('tomato')
        else:
            # Not included in assignment
            raise ValueError(f'Unknown class in {folder_lower}')
            continue

        folder_path = os.path.join(home_folder, folder)
        file_list = sorted(os.listdir(folder_path))
        for im_file in file_list:
            
            name, _ = os.path.splitext(im_file)
            index_file = int(name.split('_')[-2])   # Second to last! Last element is width image (=100)
            if (index_file % factor_downsample) != 0:
                continue
        
            im_path = os.path.join(folder_path, im_file)

            im = plt.imread(im_path)
            im = im_preprocess(im)

            x.append(im)
            y.append(y_index)

    print(f'Finished {dataset}-dataset')
    return x, y

In [ ]:
print(class_names)

print('Training data')
x_train, y_train = get_data('training', factor_downsample = 1)
x_valid, y_valid = get_data('validation', factor_downsample = 1)
x_test, y_test = get_data('testing', factor_downsample = 1)

# Training

In [ ]:
x_train = np.stack(x_train, axis=0).astype(np.float16)
x_valid = np.stack(x_valid, axis=0).astype(np.float16)
x_test = np.stack(x_test, axis=0).astype(np.float16)

y_train = np.stack(y_train, axis=0).astype(np.int16)
y_valid = np.stack(y_valid, axis=0).astype(np.int16)
y_test = np.stack(y_test, axis=0).astype(np.int16)


index = [i for i in range(len(x_train))]
np.random.seed(10)
np.random.shuffle(index)
selected_index = index[0: int(len(index)/2)]
x_train = x_train[selected_index,:,:,:]
y_train = y_train[selected_index]

index_valid = [i for i in range(len(x_valid))]
np.random.seed(10)
np.random.shuffle(index_valid)
selected_index_valid = index_valid[0: int(len(index_valid)/2)]
x_valid = x_valid[selected_index_valid,:,:,:]
y_valid = y_valid[selected_index_valid]

index_test = [i for i in range(len(x_test))]
np.random.seed(10)
np.random.shuffle(index_test)
selected_index_test = index_test[0: int(len(index_test)/2)]
x_test = x_test[selected_index_test,:,:,:]
y_test = y_test[selected_index_test]



def y_to_apple(y):
    i_apple = class_names.index('apple')
    y_apple = np.equal(y, i_apple, dtype=np.int16)
    return y_apple

y_train_apple = y_to_apple(y_train)
y_valid_apple = y_to_apple(y_valid)
y_test_apple = y_to_apple(y_test)

y_train_fruit = to_categorical(y_train, num_classes=None)
y_valid_fruit = to_categorical(y_valid, num_classes=None)
y_test_fruit = to_categorical(y_test, num_classes=None)

## Building a neural network

In [12]:
pract_folder = 'drive/My Drive/AI_pract3_nn2021'
logs_folder = os.path.join(pract_folder, 'logs')
weights_folder = os.path.join(pract_folder, 'weights')
if not os.path.exists(weights_folder):
    os.makedirs(weights_folder)


In [13]:
def get_model_base_simple(k = 4):  
    model = Sequential()
    model.epoch = 0     # to save amount of epochs trained

    model.add(InputLayer((100, 100, 3)))
    model.add(AveragePooling2D(pool_size=(2, 2))) # downsampling
    model.add(Conv2D(k, (3, 3), activation='elu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(k, (3, 3), activation='elu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(k, (4, 4), activation='elu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(k, activation='elu'))

    return model


def get_model_apple_simple(lr, loss='mse'):
  
    model = get_model_base_simple()

    # single class classification
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=loss,
                optimizer=SGD(lr),
                metrics=['accuracy', precision, recall])

    return model


def get_model_fruit_simple(lr):
    model = get_model_base_simple()
  
    ### To implement yourself
    model.add(None)
    model.compile(loss=None,
                optimizer=SGD(lr),
                metrics=['accuracy'])

    return model


## Apples or not apples

In [ ]:
model_apple = get_model_apple_simple(lr=1e-2)

### uncomment if you already have a trained network
# model_apple = keras.models.load_model(os.path.join(weights_folder, '<apple.hdf5>'))

model_apple.summary()

Training.

In [ ]:
model_apple.fit(x_train, y_train_apple, validation_data=(x_valid, y_valid_apple), batch_size=32, epochs=20)


Testing

In [ ]:
y_test_pred = model_apple.predict(x_test)

###Saving and downloading model.

Alternatively you can download it manually through: *View -> Table of contents -> Files -> Right click* **.h5-> Download*

In [17]:
path = os.path.join(weights_folder, 'apple.h5')
model_apple.save(path)

## Fruit classifcation

In [ ]:
# to implement
def get_model_fruit_simple():
  return None

model_fruit = get_model_fruit_simple()

model_fruit.summary()

In [ ]:
# train

# Testing on own data

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import cv2

In [ ]:
url = 'https://images-na.ssl-images-amazon.com/images/I/319J7YpfyNL.jpg'

def open_image(path):

    response = requests.get(path)
    img = np.array(Image.open(BytesIO(response.content)))

    resize = cv2.resize(img, (100, 100))

    plt.figure()
    plt.imshow(resize)
    plt.plot()

    single_x = np.stack([resize], axis=0)
    im_resize = im_preprocess(single_x)

    # TODO preprocessing 
    ...

    return im_resize

single_x = open_image(url) # '<your_fruit.jpg>'

model_apple = load_model(weights_folder + '/<model_apple>.hdf5')
model_fruit = load_model(weights_folder + '/<model_fruit>.hdf5')

pred_apple = model_apple.predict(single_x)
pred_fruit = model_fruit.predict(single_x)

print(pred_apple)
print(pred_fruit)